* 多分类问题概率分布

$$
q_{i}= \begin{cases} 1 & \text { if } i=y \\ 0 & \text { otherwise }\end{cases}
$$

* 进行label smoothing后的概率分布(其中$\varepsilon$为一个很小的常数,即平滑系数)

$$
q_{i}= \begin{cases}1-\varepsilon & \text { if } i=y \\ \varepsilon /(K-1) & \text { otherwise }\end{cases}
$$

理解: 将真实标签减去一个很小的数,然后平均分配到其他类上,实现了标签软化

***

| Predicted prob(p)    | 0.3  | 0.6  | 0.001 | 0.001 | 0.05 | 0.03   |
| -------------------- | ---- | ---- | ----- | ----- | ---- | ------ |
| **Label one-hot(y)** | 0    | 1    | 0     | 0     | 0    | 0      |
| **Label name:**      | 飞机 | 鸟   | 猫    | 狗    | 汽车 | 拖拉机 |

缺点(one-hot只是对真实情况的一种简化):
* 真实标签跟其他标签之间的关系被忽略了,很多有用的知识无法学到;比如:"鸟"和"飞机"本来也比较像,因此如果模型预测觉得二者更接近,那么应该给予更小的loss
* 倾向于让模型更加"武断",成为一个"非黑即白"的模型,导致泛化性能差
* 面对易混淆的分类任务、有噪音(误打标)的数据集时,更容易受影响


| Predicted prob(p)    | 0.3  | 0.6  | 0.001 | 0.001 | 0.05 | 0.03   |
| -------------------- | ---- | ---- | ----- | ----- | ---- | ------ |
| **Label one-hot(y)** | 0    | 1    | 0     | 0     | 0    | 0      |
| **Smoothed label(y)** | 0.05    | 0.75    | 0.05    | 0.05     | 0.05   | 0.05     |
| **Label name:**      | 飞机 | 鸟   | 猫    | 狗    | 汽车 | 拖拉机 |

***

Label Smoothing的优势(可能增强了模型泛化能力):
* 一定程度上,可以缓解模型过于武断的问题,也有一定的抗噪能力
* 提供了训练数据中类别之间的关系(数据增强)

Label Smoothing的劣势:
* 单纯地添加随机噪音,也无法反映标签之间的关系,因此对模型的提升有限,甚至有欠拟合的风险
* 它对构建将来作为教师的网络没有用处,hard目标训练将产生一个更好的教师神经网络

In [8]:
import torch.nn as nn
import torch.nn.functional as F


class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, eps=0.1, reduction='mean', ignore_index=-100):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.eps = eps
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, input, target):
        c = input.size()[-1]  # 类别数
        log_preds = F.log_softmax(input, dim=-1)
        if self.reduction == 'sum':
            loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=-1)
            if self.reduction == 'mean':
                loss = loss.mean()
        # 交叉熵损失
        cross_entropy_loss = F.nll_loss(log_preds, target, reduction=self.reduction, ignore_index=self.ignore_index)
        return loss * self.eps / c + (1 - self.eps) * cross_entropy_loss

In [9]:
import torch

x_input = torch.arange(12, dtype=torch.float32).reshape(4, 3)
y_target = torch.tensor([0, 2, 1, 2])

print(nn.CrossEntropyLoss()(x_input, y_target))
print(LabelSmoothingCrossEntropy(eps=0)(x_input, y_target))  # eps等于0时退化为交叉熵损失

print(LabelSmoothingCrossEntropy()(x_input, y_target))
print(LabelSmoothingCrossEntropy(reduction='sum')(x_input, y_target))
print(LabelSmoothingCrossEntropy(reduction='none')(x_input, y_target))

tensor(1.1576)
tensor(1.1576)
tensor(1.1826)
tensor(4.7304)
tensor([2.3076, 0.5076, 1.4076, 0.5076])
